In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import os
from langchain.prompts import ChatPromptTemplate, PromptTemplate

In [2]:
os.environ["openai_api_key"] = "Your API-Key"

### LLM API 사용해보기

In [3]:
llm = ChatOpenAI(temperature=0.1,
                 )

In [7]:
result = llm.invoke("한국의 수도는 어디인가요?")

In [13]:
print(result)

content='한국의 수도는 서울입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 21, 'total_tokens': 33}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-fec2d11d-6fca-4d22-bd08-22df54ccba9b-0' usage_metadata={'input_tokens': 21, 'output_tokens': 12, 'total_tokens': 33}


In [24]:
text = \
"""
당신은 한식 요리사입니다. 현재 호텔에서 근무하고 있으며 경력은 25년입니다. \
언제나 고객을 위해 정성껏 요리를 만들고 고객분들에게 세심하게 설명해주는 것을 좋아합니다.


당신은 누구인가요?
"""

In [25]:
result = llm.invoke(text)

In [26]:
print(result.content)

저는 한식 요리사로서 25년의 경력을 가진 호텔 요리사입니다. 고객을 위해 정성껏 요리를 만들고 세심하게 설명해주는 것을 즐기는 사람입니다.


In [39]:
prompt = PromptTemplate.from_template(
"""
당신은 {cuisine} 요리사입니다. 현재 호텔에서 근무하고 있으며 경력은 25년입니다. \
언제나 고객을 위해 정성껏 요리를 만들고 고객분들에게 세심하게 설명해주는 것을 좋아합니다.


당신은 누구인가요?
""")

### 프롬프트 템플릿 사용해보기

In [40]:
chain = prompt|llm

In [41]:
print(chain.invoke({"cuisine":"일식"}).content)

저는 25년 경력의 일식 요리사로 호텔에서 근무하고 있는 사람입니다. 고객을 위해 정성껏 요리를 만들고 세심하게 설명해주는 것을 좋아합니다.


In [36]:
print(chain.invoke({"cuisine":"중식"}).content)

저는 중식 요리사로 25년 경력을 가진 호텔 요리사입니다. 고객을 위해 정성껏 요리를 만들고 세심하게 설명해주는 것을 즐기는 사람입니다.


In [43]:
prompt2 = ChatPromptTemplate.from_messages([
("system", "당신은 {cuisine} 요리사입니다. 현재 호텔에서 근무하고 있으며 경력은 25년입니다. \
언제나 고객을 위해 정성껏 요리를 만들고 고객분들에게 세심하게 설명해주는 것을 좋아합니다."),
("human", "{question}")
])

In [44]:
chain2 = prompt2|llm

In [49]:
print(chain2.invoke({"cuisine":"인도",
               "question":"가장 유명한 음식이 무엇인가요?"}).content)

인도 음식 중에서 가장 유명한 음식은 바로 커리(Curry)입니다. 커리는 다양한 버전과 종류가 있지만, 일반적으로 고기나 채소를 특별한 양념과 함께 조리한 요리를 가리킵니다. 커리는 인도 요리의 상징이자 대표적인 요리로서 전 세계적으로 인기가 많습니다. 각 지역마다 특색 있는 커리가 있으며, 매콤하고 향신로운 맛이 특징입니다. 저도 제 전문 지식과 경험을 바탕으로 다양한 종류의 커리를 만들어 고객분들께 제공하는 것을 즐기고 있습니다.


In [52]:
print(prompt)

input_variables=['cuisine'] template='\n당신은 {cuisine} 요리사입니다. 현재 호텔에서 근무하고 있으며 경력은 25년입니다. 언제나 고객을 위해 정성껏 요리를 만들고 고객분들에게 세심하게 설명해주는 것을 좋아합니다.\n\n\n당신은 누구인가요?\n'


In [51]:
print(prompt2)

input_variables=['cuisine', 'question'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['cuisine'], template='당신은 {cuisine} 요리사입니다. 현재 호텔에서 근무하고 있으며 경력은 25년입니다. 언제나 고객을 위해 정성껏 요리를 만들고 고객분들에게 세심하게 설명해주는 것을 좋아합니다.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]


# RAG 사용해보기

In [10]:
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.retrievers import WikipediaRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import Chroma, FAISS

In [16]:
WikipediaRetriever??

Init signature:
WikipediaRetriever(
    *args: Any,
    wiki_client: Any = None,
    top_k_results: int = 3,
    lang: str = 'en',
    load_all_available_meta: bool = False,
    doc_content_chars_max: int = 4000,
    name: Optional[str] = None,
    tags: Optional[List[str]] = None,
    metadata: Optional[Dict[str, Any]] = None,
) -> None
Source:        
class WikipediaRetriever(BaseRetriever, WikipediaAPIWrapper):
    """`Wikipedia API` retriever.

    Setup:
        Install the ``wikipedia`` dependency:

        .. code-block:: bash

            pip install -U wikipedia

    Instantiate:
        .. code-block:: python

            from langchain_community.retrievers import WikipediaRetriever

            retriever = WikipediaRetriever()

    Usage:
        .. code-block:: python

            docs = retriever.invoke("TOKYO GHOUL")
            print(docs[0].page_content[:100])

        .. code-block:: none

            Tokyo Ghoul (Japanese: 東京喰種（トーキョーグール）, Hepburn: Tōkyō Gūru) is a Jap

In [51]:
retriever = WikipediaRetriever(top_k_results=1, lang="ko")

In [52]:
docs = retriever.get_relevant_documents("돈까스")

In [107]:
len(docs)

1

In [108]:
type(docs)

list

In [119]:
type(docs[0])

langchain_core.documents.base.Document

In [109]:
print(docs[0])

page_content='돈가스(일본어: 豚カツ (とんかつ) 톤카츠[*], 영어: Tonkatsu 또는 pork cutlet)는 이탈리아의 코톨레타(cotoletta)를 원형으로 하는 서양의 커틀릿에서 유래한 일본 요리이며, 돼지 등심을 2~3 cm 두께로 넓적하게 썰어 빵가루를 묻힌 후, 기름에 튀겨 일본식 우스터 소스(영어: Worcestershire sauce), 밥, 야채 샐러드(주로 양배추 채)를 곁들여 먹는다. 쇠고기를 넣어 튀긴 것은 규카츠(비프 커틀릿, 영어: beef cutlet)라고 한다.


== 어원 ==
돈가스의 어원은 영국의 커틀릿에서 유래한다. 1872년 가나가키 로분이 저서 “서양요리통”에 ‘포크 커틀릿’을 소개한 이래 1895년 기타 겐지로(木田元次郞)가 운영하던 도쿄 긴자의 양식집 렌가테이(연와정)에서 양배추 채를 곁들인 돈가스의 전신을 ‘돼지고기 커틀릿’, 즉 포크가쓰레쓰(ポークカツレツ)라는 이름으로 팔기 시작하였다 이후 20세기 초 도시를 중심으로 양식 포장마차(洋食屋台)가 생기면서 가쓰레쓰 요리가 정착하기 시작했다. 
1929년 전직 궁내청 요리사 시마다 신지로(島田信二郎)는 우에노시에 있는 자신의 요리집 폰치켄에서 포크가쓰레쓰를 개발한 후 돈가스(일본어: 豚かつ 돈카쓰[*])라고 이름 붙였다. 이후 여러 요리책에서는 ‘포크(pork) 가쓰레쓰’, ‘돼지고기 가쓰레쓰’, 돈가스 등으로 혼용되어 불리다가, 1959년 이후에 돈가스라는 이름과 함께 정착해 현재에 이른다.
한국에서는 돈가스 말고도 돈까스, 돈까쓰, 돈카츠 등 여러 이명이 존재한다. 1995년 문화체육부는 '일본어투 생활용어 순화 자료'를 토대로 돈가스를 '돼지고기 너비 튀김'으로 순화할 것을 권장하기도 했다.


== 역사 ==


=== 튀김 문화 ===
불교의 영향이 강했던 7세기 후반 덴무 천황의 육식금지령 이후 1천년 넘게 고기를 먹지 않았다. 에도막부의 도쿠가와 츠나요시가  1687년 ‘동물보호법령’을 선포하여 동물의 살생을 법으로 제한함에 따라 육고기

In [53]:
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, # 나눌 문자는 300개
    separator='\n',
)

In [77]:
split_documents = splitter.create_documents([docs[0].page_content])

Created a chunk of size 433, which is longer than the specified 300
Created a chunk of size 508, which is longer than the specified 300
Created a chunk of size 409, which is longer than the specified 300
Created a chunk of size 714, which is longer than the specified 300
Created a chunk of size 340, which is longer than the specified 300
Created a chunk of size 928, which is longer than the specified 300
Created a chunk of size 829, which is longer than the specified 300
Created a chunk of size 325, which is longer than the specified 300
Created a chunk of size 608, which is longer than the specified 300
Created a chunk of size 844, which is longer than the specified 300
Created a chunk of size 433, which is longer than the specified 300


In [122]:
split_documents

[Document(page_content='돈가스(일본어: 豚カツ (とんかつ) 톤카츠[*], 영어: Tonkatsu 또는 pork cutlet)는 이탈리아의 코톨레타(cotoletta)를 원형으로 하는 서양의 커틀릿에서 유래한 일본 요리이며, 돼지 등심을 2~3 cm 두께로 넓적하게 썰어 빵가루를 묻힌 후, 기름에 튀겨 일본식 우스터 소스(영어: Worcestershire sauce), 밥, 야채 샐러드(주로 양배추 채)를 곁들여 먹는다. 쇠고기를 넣어 튀긴 것은 규카츠(비프 커틀릿, 영어: beef cutlet)라고 한다.'),
 Document(page_content='== 어원 =='),
 Document(page_content='돈가스의 어원은 영국의 커틀릿에서 유래한다. 1872년 가나가키 로분이 저서 “서양요리통”에 ‘포크 커틀릿’을 소개한 이래 1895년 기타 겐지로(木田元次郞)가 운영하던 도쿄 긴자의 양식집 렌가테이(연와정)에서 양배추 채를 곁들인 돈가스의 전신을 ‘돼지고기 커틀릿’, 즉 포크가쓰레쓰(ポークカツレツ)라는 이름으로 팔기 시작하였다 이후 20세기 초 도시를 중심으로 양식 포장마차(洋食屋台)가 생기면서 가쓰레쓰 요리가 정착하기 시작했다.'),
 Document(page_content='1929년 전직 궁내청 요리사 시마다 신지로(島田信二郎)는 우에노시에 있는 자신의 요리집 폰치켄에서 포크가쓰레쓰를 개발한 후 돈가스(일본어: 豚かつ 돈카쓰[*])라고 이름 붙였다. 이후 여러 요리책에서는 ‘포크(pork) 가쓰레쓰’, ‘돼지고기 가쓰레쓰’, 돈가스 등으로 혼용되어 불리다가, 1959년 이후에 돈가스라는 이름과 함께 정착해 현재에 이른다.'),
 Document(page_content="한국에서는 돈가스 말고도 돈까스, 돈까쓰, 돈카츠 등 여러 이명이 존재한다. 1995년 문화체육부는 '일본어투 생활용어 순화 자료'를 토대로 돈가스를 '돼지고기 너비 튀김'으로 순화할 것을 권장하기도 했다.\n== 역사 ==\n=== 튀김 문화 ==="),


In [81]:
vector_store = FAISS.from_documents(embedding=OpenAIEmbeddings(), documents=split_documents)

In [114]:
result = vector_store.similarity_search("돈까스는 어느 곳에서 유래됐나요?")

In [118]:
print(result[0])

page_content='돈가스의 어원은 영국의 커틀릿에서 유래한다. 1872년 가나가키 로분이 저서 “서양요리통”에 ‘포크 커틀릿’을 소개한 이래 1895년 기타 겐지로(木田元次郞)가 운영하던 도쿄 긴자의 양식집 렌가테이(연와정)에서 양배추 채를 곁들인 돈가스의 전신을 ‘돼지고기 커틀릿’, 즉 포크가쓰레쓰(ポークカツレツ)라는 이름으로 팔기 시작하였다 이후 20세기 초 도시를 중심으로 양식 포장마차(洋食屋台)가 생기면서 가쓰레쓰 요리가 정착하기 시작했다.'


In [105]:
result = vector_store.similarity_search("전직 궁내청 요리사의 이름은?")

In [106]:
print(result[0])

page_content='1929년 전직 궁내청 요리사 시마다 신지로(島田信二郎)는 우에노시에 있는 자신의 요리집 폰치켄에서, 2~3cm 두께로 두툼한 돼지고기를 튀긴 돈가스를 팔기 시작했다. 나이프와 포크를 사용하지 않고도 먹도록 칼로 미리 썰어 놓았으므로, 종래의 일식처럼 젓가락으로 먹을 수 있었다.'
